# SMM Parallel
This notebook shows how to save an optimization to disk, load it, and restart the algorithm n addional steps.

## Preliminaries

In [4]:
using SMM
using DataStructures
using DataFrames
using Plots

srand(1234);

# Define true values of parameters
#---------------------------------
trueValues = OrderedDict("mu1" => [-1.0], "mu2" => [1.0])


DataStructures.OrderedDict{String,Array{Float64,1}} with 2 entries:
  "mu1" => [-1.0]
  "mu2" => [1.0]

## Set up a SMM problem

In [9]:
#------------------------------------------------
# Options
#-------------------------------------------------
# Boolean: do you want to save the plots to disk?
savePlots = false

#------------------------
# initialize the problem:
#------------------------

# Specify the initial values for the parameters, and their support:
pb = OrderedDict("p1" => [0.2,-3,3] , "p2" => [-0.2,-2,2])

# Specify moments to be matched + subjective weights:
moms = DataFrame(name=["mu1","mu2"],value=[trueValues["mu1"][], trueValues["mu2"][]], weight=ones(2))

# GMM objective function to be minized.
# It returns a weigthed distance between empirical and simulated moments
@everywhere function objfunc_normal(ev::Eval; verbose = false)

    start(ev)


    # when running in parallel, display worker's id:
    #-----------------------------------------------
    if verbose == true
        if nprocs() > 1
          println(myid())
        end
    end

    # extract parameters from ev:
    #----------------------------
    mu  = collect(values(ev.params))

    # compute simulated moments
    #--------------------------
    # Monte-Carlo:
    #-------------
    ns = 100000 #number of i.i.d draws from N([mu], sigma)
    #initialize a multivariate normal N([mu], sigma)
    #sigma is set to be the identity matrix
    sigma = [1.0; 1.0]
    # draw ns observations from N([mu], sigma):
    randMultiNormal = SMM.MvNormal(mu,SMM.PDiagMat(sigma))
    # calculate the mean of the simulated data
    # let's "freeze" the random number generated between each draw of parameters
    srand(1234)
    simM            = mean(rand(randMultiNormal,ns),2)
    # store simulated moments in a dictionary
    simMoments = Dict(:mu1 => simM[1], :mu2 => simM[2])

    # Calculate the weighted distance between empirical moments
    # and simulated ones:
    #-----------------------------------------------------------
    v = Dict{Symbol,Float64}()
    for (k, mom) in dataMomentd(ev)
        # If weight for moment k exists:
        #-------------------------------
        if haskey(SMM.dataMomentWd(ev), k)
            # divide by weight associated to moment k:
            #----------------------------------------
            v[k] = ((simMoments[k] .- mom) ./ SMM.dataMomentW(ev,k)) .^2
        else
            v[k] = ((simMoments[k] .- mom) ) .^2
        end
    end

    # Set value of the objective function:
    #------------------------------------
    setValue(ev, mean(collect(values(v))))

    # also return the moments
    #-----------------------
    setMoment(ev, simMoments)

    # flag for success:
    #-------------------
    ev.status = 1

    # finish and return
    finish(ev)

    return ev
end



# Initialize an empty MProb() object:
#------------------------------------
mprob = MProb()

# Add structural parameters to MProb():
# specify starting values and support
#--------------------------------------
addSampledParam!(mprob,pb)

# Add moments to be matched to MProb():
#--------------------------------------
addMoment!(mprob,moms)

# Attach an objective function to MProb():
#----------------------------------------
addEvalFunc!(mprob, objfunc_normal)

objfunc_normal (generic function with 1 method)

## Saving Options

In order to save the optimization at every `n` steps, it is necessary to include the following items in the dictionary containing the optimization options:
* "save_frequency": the optimization will be saved every ̀save_frequency steps
* "filename": the name under which the optimization will be saved
Independently to the value of `save_frequency`, The optimization will be saved to disk at the final iteration. If `filename` is not specified, a random name will be generated. 

In [10]:
# estimation options:
#--------------------
# number of iterations for each chain
niter = 50
# number of chains
nchains = 3

opts = Dict("N"=>nchains,
        "maxiter"=>niter,
        "maxtemp"=> 5,
        "coverage"=>0.025,
        "sigma_update_steps"=>10,
        "sigma_adjust_by"=>0.01,
        "smpl_iters"=>1000,
        "parallel"=>true,
        "maxdists"=>[0.05 for i in 1:nchains],
        "mixprob"=>0.3,
        "acc_tuner"=>12.0,
        "animate"=>false,
        "save_frequency"=>10,
        "filename"=>"MyMAProblem")


#---------------------------------------
# Let's set-up and run the optimization
#---------------------------------------
# set-up BGP algorithm:
MA = MAlgoBGP(mprob,opts)

# run the estimation:
@time SMM.run!(MA)

# show a summary of the optimization:
@show SMM.summary(MA)

11:34:43:INFO:Main:Starting estimation loop.
11:34:48:INFO:Main:saved data at iteration 10
11:34:52:INFO:Main:saved data at iteration 20
11:34:57:INFO:Main:saved data at iteration 30
11:35:01:INFO:Main:saved data at iteration 40
11:35:06:INFO:Main:saved data at iteration 50
11:35:06:INFO:Main:Done with estimation after 0.4 minutes


3×5 DataFrames.DataFrame
│ Row │ id │ acc_rate  │ perc_exchanged │ exchanged_most_with │ best_val │
├─────┼────┼───────────┼────────────────┼─────────────────────┼──────────┤
│ 1   │ 1  │ 0.26      │ 0.0            │ 0                   │ 1.23645  │
│ 2   │ 2  │ 0.0930233 │ 14.0           │ 3                   │ 0.695447 │
│ 3   │ 3  │ 0.116279  │ 14.0           │ 2                   │ 0.695447 │

## Loading and restarting 

To load an optimization, use the function `readMalgo`

In [11]:
MA2 = readMalgo(opts["filename"])
@show SMM.summary(MA2)

3×5 DataFrames.DataFrame
│ Row │ id │ acc_rate  │ perc_exchanged │ exchanged_most_with │ best_val │
├─────┼────┼───────────┼────────────────┼─────────────────────┼──────────┤
│ 1   │ 1  │ 0.26      │ 0.0            │ 0                   │ 1.23645  │
│ 2   │ 2  │ 0.0930233 │ 14.0           │ 3                   │ 0.695447 │
│ 3   │ 3  │ 0.116279  │ 14.0           │ 2                   │ 0.695447 │

 23.280594 seconds (116.06 k allocations: 237.325 MiB, 93.87% gc time)
SMM.summary(MA) = 3×5 DataFrames.DataFrame
│ Row │ id │ acc_rate  │ perc_exchanged │ exchanged_most_with │ best_val │
├─────┼────┼───────────┼────────────────┼─────────────────────┼──────────┤
│ 1   │ 1  │ 0.26      │ 0.0            │ 0                   │ 1.23645  │
│ 2   │ 2  │ 0.0930233 │ 14.0           │ 3                   │ 0.695447 │
│ 3   │ 3  │ 0.116279  │ 14.0           │ 2                   │ 0.695447 │
SMM.summary(MA2) = 3×5 DataFrames.DataFrame
│ Row │ id │ acc_rate  │ perc_exchanged │ exchanged_most_with │ best_val │
├─────┼────┼───────────┼────────────────┼─────────────────────┼──────────┤
│ 1   │ 1  │ 0.26      │ 0.0            │ 0                   │ 1.23645  │
│ 2   │ 2  │ 0.0930233 │ 14.0           │ 3                   │ 0.695447 │
│ 3   │ 3  │ 0.116279  │ 14.0           │ 2                   │ 0.695447 │


To restart the optimization, simply use the function `restart!`

In [12]:
# re-start the estimation adding newiters steps
#----------------------------------------------
newiters = 50
# re-set seed to keep randomness constant
# otherwise results are not comparable
srand(1234)
restart!(MA2, newiters)

# show a summary of the optimization:
@show SMM.summary(MA2)

11:35:33:INFO:Main:Restarting estimation loop with 50 iterations.
11:35:33:INFO:Main:Current best value on chain 1 before restarting 1.23644627881224
11:35:33:INFO:Main:saved data at iteration 50
11:35:38:INFO:Main:saved data at iteration 60
11:35:42:INFO:Main:saved data at iteration 70
11:35:47:INFO:Main:saved data at iteration 80
11:35:52:INFO:Main:saved data at iteration 90
11:35:57:INFO:Main:saved data at iteration 100
11:35:57:INFO:Main:Done with estimation after 0.4 minutes
11:35:57:INFO:Main:New best value on chain 1 = 1.23644627881224


3×5 DataFrames.DataFrame
│ Row │ id │ acc_rate  │ perc_exchanged │ exchanged_most_with │ best_val │
├─────┼────┼───────────┼────────────────┼─────────────────────┼──────────┤
│ 1   │ 1  │ 0.16      │ 0.0            │ 0                   │ 1.23645  │
│ 2   │ 2  │ 0.0537634 │ 7.0            │ 3                   │ 0.695447 │
│ 3   │ 3  │ 0.0537634 │ 7.0            │ 2                   │ 0.695447 │

SMM.summary(MA2) = 3×5 DataFrames.DataFrame
│ Row │ id │ acc_rate  │ perc_exchanged │ exchanged_most_with │ best_val │
├─────┼────┼───────────┼────────────────┼─────────────────────┼──────────┤
│ 1   │ 1  │ 0.16      │ 0.0            │ 0                   │ 1.23645  │
│ 2   │ 2  │ 0.0537634 │ 7.0            │ 3                   │ 0.695447 │
│ 3   │ 3  │ 0.0537634 │ 7.0            │ 2                   │ 0.695447 │


In [13]:
# re-start the estimation with MA
#---------------------------------
newiters = 50
srand(1234)
restart!(MA, newiters)

# show a summary of the optimization:
@show SMM.summary(MA)

11:36:20:INFO:Main:Restarting estimation loop with 50 iterations.
11:36:20:INFO:Main:Current best value on chain 1 before restarting 1.23644627881224
11:36:20:INFO:Main:saved data at iteration 50
11:36:26:INFO:Main:saved data at iteration 60
11:36:31:INFO:Main:saved data at iteration 70
11:36:35:INFO:Main:saved data at iteration 80
11:36:40:INFO:Main:saved data at iteration 90
11:36:44:INFO:Main:saved data at iteration 100
11:36:45:INFO:Main:Done with estimation after 0.4 minutes
11:36:45:INFO:Main:New best value on chain 1 = 1.23644627881224


3×5 DataFrames.DataFrame
│ Row │ id │ acc_rate  │ perc_exchanged │ exchanged_most_with │ best_val │
├─────┼────┼───────────┼────────────────┼─────────────────────┼──────────┤
│ 1   │ 1  │ 0.16      │ 0.0            │ 0                   │ 1.23645  │
│ 2   │ 2  │ 0.0537634 │ 7.0            │ 3                   │ 0.695447 │
│ 3   │ 3  │ 0.0537634 │ 7.0            │ 2                   │ 0.695447 │

In [14]:
# New estimation, with (niter + newiters) iterations
# without starting and restarting
#---------------------------------------------------
# set-up BGP algorithm:
MA3 = MAlgoBGP(mprob,opts)

# run the estimation:
srand(1234)
@time SMM.run!(MA3)

# show a summary of the optimization:
@show SMM.summary(MA3)

11:36:45:INFO:Main:Starting estimation loop.


SMM.summary(MA) = 3×5 DataFrames.DataFrame
│ Row │ id │ acc_rate  │ perc_exchanged │ exchanged_most_with │ best_val │
├─────┼────┼───────────┼────────────────┼─────────────────────┼──────────┤
│ 1   │ 1  │ 0.16      │ 0.0            │ 0                   │ 1.23645  │
│ 2   │ 2  │ 0.0537634 │ 7.0            │ 3                   │ 0.695447 │
│ 3   │ 3  │ 0.0537634 │ 7.0            │ 2                   │ 0.695447 │


11:36:49:INFO:Main:saved data at iteration 10
11:36:54:INFO:Main:saved data at iteration 20
11:36:59:INFO:Main:saved data at iteration 30
11:37:03:INFO:Main:saved data at iteration 40
11:37:08:INFO:Main:saved data at iteration 50
11:37:13:INFO:Main:saved data at iteration 60
11:37:17:INFO:Main:saved data at iteration 70
11:37:22:INFO:Main:saved data at iteration 80
11:37:26:INFO:Main:saved data at iteration 90
11:37:31:INFO:Main:saved data at iteration 100
11:37:31:INFO:Main:Done with estimation after 0.8 minutes


3×5 DataFrames.DataFrame
│ Row │ id │ acc_rate  │ perc_exchanged │ exchanged_most_with │ best_val │
├─────┼────┼───────────┼────────────────┼─────────────────────┼──────────┤
│ 1   │ 1  │ 0.16      │ 0.0            │ 0                   │ 1.23645  │
│ 2   │ 2  │ 0.0618557 │ 3.0            │ 3                   │ 0.695447 │
│ 3   │ 3  │ 0.0515464 │ 3.0            │ 2                   │ 0.702253 │

 46.570475 seconds (252.23 k allocations: 475.540 MiB, 93.75% gc time)
SMM.summary(MA3) = 3×5 DataFrames.DataFrame
│ Row │ id │ acc_rate  │ perc_exchanged │ exchanged_most_with │ best_val │
├─────┼────┼───────────┼────────────────┼─────────────────────┼──────────┤
│ 1   │ 1  │ 0.16      │ 0.0            │ 0                   │ 1.23645  │
│ 2   │ 2  │ 0.0618557 │ 3.0            │ 3                   │ 0.695447 │
│ 3   │ 3  │ 0.0515464 │ 3.0            │ 2                   │ 0.702253 │


## Comparing the results

MA, MA2 and MA3 should give the same results (once we control for randomness).
Let's compare the estimates we obtain from the first chain.

In [19]:
for (algo, algoName) in zip([MA, MA2, MA3], ["MA", "MA2", "MA3"])
    # Realization of the first chain:
    #-------------------------------
    dat_chain1 = SMM.history(algo.chains[1])

    # discard the first 10th of the observations ("burn-in" phase):
    #--------------------------------------------------------------
    dat_chain1[round(Int, niter/10):niter, :]

    # keep only accepted draws:
    #--------------------------
    dat_chain1 = dat_chain1[dat_chain1[:accepted ].== true, : ]

    # create a list with the parameters to be estimated
    parameters = [Symbol(String("mu$(i)")) for i=1:2]
    # list with the corresponding priors:
    #------------------------------------
    estimatedParameters = [Symbol(String("p$(i)")) for i=1:2]


    # Quasi Posterior mean and quasi posterior median for each parameter:
    #-------------------------------------------------------------------
    for (estimatedParameter, param) in zip(estimatedParameters, parameters)

      println("Algo = $(algoName)")
      println("Quasi posterior mean for $(String(estimatedParameter)) = $(mean(dat_chain1[estimatedParameter]))")
      println("Quasi posterior median for $(String(estimatedParameter)) = $(median(dat_chain1[estimatedParameter]))")

    end
end

Algo = MA
Quasi posterior mean for p1 = 0.6672440792426643
Quasi posterior median for p1 = 0.6587775067143288
Algo = MA
Quasi posterior mean for p2 = -0.06854953042519724
Quasi posterior median for p2 = -0.10794430374357472
Algo = MA2
Quasi posterior mean for p1 = 0.6672440792426643
Quasi posterior median for p1 = 0.6587775067143288
Algo = MA2
Quasi posterior mean for p2 = -0.06854953042519724
Quasi posterior median for p2 = -0.10794430374357472
Algo = MA3
Quasi posterior mean for p1 = 0.6761660814751674
Quasi posterior median for p1 = 0.6587775067143288
Algo = MA3
Quasi posterior mean for p2 = -0.10727907757573654
Quasi posterior median for p2 = -0.10794430374357472


Estimates are the same, as expected.